In [1]:
#add nchar greater than 40

import spacy
import re
import pandas as pd

nlp = spacy.load('en_core_web_sm')

def validate_data(df):
    df = df[~df['text'].str.match(r'hon\.$')]
    df = df[~df['text'].str.match(r'^(?![A-Z])')]
    return df

def validate_triples_structure(doc):
    for token in doc:
        if 'VERB' and 'NOUN' in token.pos_:
            return doc

def leftward_sent(doc): # extract leftward sentences without wh- movement 
    if len(doc) > 20:
        #return doc if doc[1].pos_ == 'VERB' else 'NaN'
        if doc[0].pos_ != 'NOUN' and doc[1].pos_ == 'VERB': 
            return 'leftward_sent'

def interrogative_sent(doc): # extract interrogative sentences (with wh- movement)
    if len(doc) > 20:
        #return doc if doc.match(r'^Which (.*)\?$|^What (.*)\?$', re.IGNORECASE) else 'NaN'
        if doc.match(r'^Which (.*)\?$|^What (.*)\?$', re.IGNORECASE):
            return 'interrogative_sent'

def comp_sent(doc): # extract compound, complex, compound-complex sentences
    if len(doc) > 40:
        #return doc if doc.match(r' and,| but,| or,') else 'NaN'
        if doc.match(r' and,| but,| or,'):
            return 'comp_sent'

def export_sentences(import_dir):
    df = pd.read_csv(import_dir)



export_sentences('~/hansard.csv')


/home/stephbuon/.local/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/stephbuon/.local/lib/python3.9/site-packages/spacy/util.py:837: UserWarning: [W095] Model 'en_core_web_sm' (3.2.0) was trained with spaCy v3.2 and may not be 100% compatible with the current version (3.3.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [ ]:
hansard = pd.read_csv('~/hansard.csv')
hansard = hansard[hansard['text']].copy()

hansard = validate_data(hansard)

hansard['text'] = list(nlp.pipe(hansard['text'], disable = ['ent']))

hansard['tag'] = hansard['text'].apply(leftward_sent)
hansard['tag'] = hansard['text'].apply(interrogative_sent)
hansard['tag'] = hansard['text'].apply(comp_sent)

types = ['leftward_sent', 'interrogative_sent', 'comp_sent']
out = pd.DataFrame()
for type in types:
    out = hansard[hansard['tag'] == type].sample(1000)
    out.to_csv('~/' + type + '.csv')

hansard = hansard.sample(n = 1000)
hansard.to_csv('~/random_sent.csv')